## Dataset "Películas" en Pandas

### Importación modulos a utilizar

* `import os`: Se importa este módulo para la llamada a las rutas y directorios que existen en el sistema. 

* `import pandas`: Se importa este módulo para realizar el procesamiento del archivo csv, y así generar uno nuevo con la información necesaria.  

* `import random`: Se importa de este módulo la función 'sample'. Función que retorna una lista con una cantidad especificada de items, seleccionados de manera aleatoria, de una secuencia.   

* `import itertools`: Se importa este módulo para obtener de la lista de listas de palabras de los overviews, una única lista con los elementos de cada una.  

* `import string`: En particular, se importa la variable 'puntuaction', la cual contiene todos los caracteres especiales.  

* `import Counter`: Se importa el módulo para la obtención de los 100 valores más repetidos de una secuencia de items.   

In [3]:
import pandas as pd
import os
import random
from itertools import chain
from string import punctuation
from collections import Counter

### Definición de funciones necesarias

* Función `elijo_palabras`: usada para elegir 3 palabras aleatorias de los overviews de las películas

In [4]:
def elijo_palabras(secuencia):
    """Retorna una lista de hasta 3 items elegidos al azar de una lista"""
    if len(secuencia)<=2:
        return secuencia
    else:
        return random.sample(secuencia,3)

* Función `not_top_x`: Recibe una lista de palabras y descarte aquellas que no estan en el top_x. Utilizada para la obtención de aquellas palabras de los overviews que no se encuentran entre las más repetidas.

In [5]:
def not_top_x(top_x,lista:list[str]):
    """ Retorna lista de str que no se encuentran en top_x
    """
    return list(filter(lambda pal: pal not in top_x and (pal !='' and pal !='—'),set(lista)))

### Lectura del csv de películas

Se genera un dataframe con información a partir del archivo csv. Se toma del mismo, las columnas a utilizar (Genre, Original_Language, Release_Date, Vote_Average, Overview, Title).
- `usecols`: lista de las columnas a tomar

In [6]:
path_csv=os.path.join(os.getcwd(),'..','csv','mymoviedb.csv')

columnas=['Genre','Original_Language','Release_Date','Vote_Average','Overview','Title']

peliculas=pd.read_csv(path_csv,usecols=columnas)

### Procesamiento del archivo

Reordenamiento de las columnas requeridas al siguiente orden: Genre, Original_Language, Release_Date, Vote_Average, Overview, Title. Dado por la lista *columnas* declarada previamente.

In [7]:
peliculas=peliculas.reindex(columns=columnas)

Descarte de las películas que no tienen “overview” y las películas cuyo idioma original tenga más de 2 caracteres

In [8]:
df=peliculas[(peliculas.Overview != '')&(peliculas.Original_Language.apply(len)<3)] 

Toma cada overviews de las peliculas y lo separa por palabras, convirtiéndolas en minúscula y quitándoles los caracteres especiales.

In [ ]:
palabras_overviews=df.Overview.apply(lambda x: x.lower().split())

for i,lista in enumerate(palabras_overviews):
    palabras_overviews.iloc[i]=list(map(lambda x:x.strip(punctuation),lista))  

A partir de las palabras obtenidas anteriormente, se agrupa en una única lista todas las palabras de todos los overviews, para luego, poder obtener las palabras más comunes.

Obtengo las 100 palabras más repetidas entre todos los overviews, a partir de la lista generada y mediante la utilización del método *most_common*.

In [10]:
palabras=list(chain.from_iterable(palabras_overviews))
top_100 = list(zip(*Counter(palabras).most_common(100)))[0]

Toma de la lista de listas de palabras de cada overview, al menos 3 palabras al azar que no se encuentran en el top 100. Generando un único string con las palabras seleccionadas. 

Para esto, se utilizan las funciones definidas *elijo_palabras* y *not_top_x*.  



In [11]:
for i,lista in enumerate(palabras_overviews):
    palabras_overviews.iloc[i]=';'.join(elijo_palabras(not_top_x(top_100,lista)))

En la columna Overview del DataFrame se encuentran ahora, las palabras elegidas

In [12]:
peliculas.Overview=palabras_overviews

Conversión a archivo csv mediante la función de pandas. 
 
Formato de archivo que será utilizado, tanto por el juego para la obtención de los datos utilizados, como para la recopilación de información generada por el mismo.

In [13]:
archivo_nuevo=os.path.join(os.getcwd(),'..','csv','data_set_peliculas.csv')
peliculas.to_csv(archivo_nuevo,index=False)